# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [2]:
with open("litw-win.txt", "r") as f:
    words = f.read().split()

def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]


def correct_typo(word, words_list):
    min_distance = float("inf")
    closest_word = word

    for w in words_list:
        distance = levenshtein_distance(word, w)
        if distance < min_distance:
            min_distance = distance
            closest_word = w

    return closest_word

corrected_text = []
for word in text.split():
    if word in words:
        corrected_text.append(word)
    else:
        corrected_text.append(correct_typo(word, words))

corrected_text = " ".join(corrected_text)
print(text)
print(corrected_text)

с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий
с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [4]:
pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=dc0b8a695284fe062af5df40c045100a58b11cf25b716afcdfb0f9b1ff86b978
  Stored in directory: c:\users\marya\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
Note: you may need to restart the kernel to use updated packages.


In [5]:
from nltk.stem.snowball import SnowballStemmer
from pymorphy2 import MorphAnalyzer

In [6]:
words = corrected_text.split()

stemmer = SnowballStemmer("russian")
morph = MorphAnalyzer()

stemmed_words = [stemmer.stem(word) for word in words]
lemmatized_words = [morph.parse(word)[0].normal_form for word in words]

print(stemmed_words)
print(lemmatized_words)

['с', 'величайш', 'усил', 'выбра', 'из', 'поток', 'убега', 'люд', 'кутуз', 'со', 'свит', 'уменьш', 'вдво', 'поеха', 'на', 'звук', 'выстрел', 'русск', 'оруд']
['с', 'великий', 'усилие', 'выбраться', 'из', 'поток', 'убегать', 'человек', 'кутузов', 'с', 'свита', 'уменьшиться', 'вдвое', 'поехать', 'на', 'звук', 'выстрел', 'русский', 'орудие']


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform([corrected_text])

print(vectorizer.get_feature_names_out())
print(X.toarray())

['вдвое' 'величайшим' 'выбравшись' 'выстрелов' 'звуки' 'из' 'кутузов'
 'людей' 'на' 'орудий' 'поехал' 'потока' 'русских' 'свитой' 'со'
 'убегающих' 'уменьшившейся' 'усилием']
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [8]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import edit_distance
import random

In [9]:
df = pd.read_csv('recipes_sample.csv')
descriptions = df['description'].tolist()

In [10]:
words = set()
for description in descriptions:
    if pd.isna(description):
        continue
    tokens = word_tokenize(description)
    words.update(tokens)

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [11]:
for i in range(5):
    word1, word2 = random.sample(words, 2)
    distance = edit_distance(word1, word2)
    print(f'Расстояние редактирования между словами "{word1}" и "{word2}" равно {distance}')

Расстояние редактирования между словами "honorable" и "pot-lucks" равно 8
Расстояние редактирования между словами "1950s" и "hates" равно 4
Расстояние редактирования между словами "collapse" и "finished" равно 8
Расстояние редактирования между словами "turban" и "fyi" равно 6
Расстояние редактирования между словами "butternut" и "parcels" равно 8


C:\Users\marya\AppData\Local\Temp\ipykernel_3952\1417854365.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  word1, word2 = random.sample(words, 2)


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [12]:
def find_closest_words(word, words, k):
    distances = [(edit_distance(word, w), w) for w in words]
    distances.sort(key=lambda x: x[0])
    closest_words = [w[1] for w in distances[:k]]
    return closest_words

In [13]:
word = "processing"
k = 5
closest_words = find_closest_words(word, words, 5)
print(f"{k} ближайших слов к '{word}': {closest_words}")

5 ближайших слов к 'processing': ['processing', 'proceeding', 'pressing', 'processor', 'preceding']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [25]:
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\marya\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [26]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

data = []
for word in words:
    stemmed_word = stemmer.stem(word)
    normalized_word = lemmatizer.lemmatize(word)
    data.append([word, stemmed_word, normalized_word])

df_words = pd.DataFrame(data, columns=['word', 'stemmed_word', 'normalized_word'])
df_words.set_index('word', inplace=True)

In [27]:
print(df_words.head(10))

                        stemmed_word    normalized_word
word                                                   
sewing                           sew             sewing
berry-and-yoghurt  berry-and-yoghurt  berry-and-yoghurt
aus/nz                        aus/nz             aus/nz
customising                 customis        customising
dillman                      dillman            dillman
away                            away               away
las                              las                 la
babyfit                      babyfit            babyfit
set-up                        set-up             set-up
mini-rolls                  mini-rol         mini-rolls


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [28]:
stop_words = set(stopwords.words('english'))

filtered_descriptions = []
for description in descriptions:
    if pd.isna(description):
        continue
    tokens = word_tokenize(description)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_description = ' '.join(filtered_tokens)
    filtered_descriptions.append(filtered_description)

total_words = sum(len(word_tokenize(description)) for description in descriptions if not pd.isna(description))
total_filtered_words = sum(len(word_tokenize(description)) for description in filtered_descriptions)
stop_words_ratio = (total_words - total_filtered_words) / total_words
print(f'Доля стоп-слов: {stop_words_ratio:.2%}')

Доля стоп-слов: 40.26%


In [29]:
all_words = [word for description in descriptions if not pd.isna(description) for word in word_tokenize(description)]
all_filtered_words = [word for description in filtered_descriptions for word in word_tokenize(description)]

top_words = Counter(all_words).most_common(10)
top_filtered_words = Counter(all_filtered_words).most_common(10)

print('Топ-10 самых часто употребляемых слов до удаления стоп-слов:')
for word, count in top_words:
    print(f'{word}: {count}')

print('Топ-10 самых часто употребляемых слов после удаления стоп-слов:')
for word, count in top_filtered_words:
    print(f'{word}: {count}')

Топ-10 самых часто употребляемых слов до удаления стоп-слов:
.: 66140
the: 40257
,: 38544
a: 35030
and: 30425
i: 27799
this: 27132
to: 23508
it: 23212
is: 20501
Топ-10 самых часто употребляемых слов после удаления стоп-слов:
.: 66233
,: 38544
!: 16054
recipe: 15121
's: 7689
make: 6367
``: 5470
time: 5198
n't: 4798
use: 4645


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

random_rows = df.sample(5)
random_descriptions = random_rows['description'].tolist()
random_names = random_rows['name'].tolist()

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(random_descriptions)
tfidf_vectors = tfidf_matrix.toarray()

for i, description in enumerate(random_descriptions):
    vector = tfidf_vectors[i]
    print(f'Описание рецепта: {description}')
    print(f'Вектор: {vector}')

Описание рецепта: this is a great hot appetizer recipe. i made this for a dinner party and everyone loved it. i have even made it using reduced fat ingredients and it is still very tasty. it is from taste of home magazine and is credited to linda wheeler.
Вектор: [0.         0.         0.34698816 0.11566272 0.         0.
 0.         0.         0.11566272 0.11566272 0.         0.11566272
 0.11566272 0.11566272 0.11566272 0.11566272 0.         0.09331595
 0.         0.11566272 0.11566272 0.11566272 0.         0.11566272
 0.46265088 0.46265088 0.         0.         0.         0.
 0.         0.11566272 0.         0.11566272 0.         0.23132544
 0.11566272 0.         0.         0.         0.         0.
 0.11566272 0.         0.         0.09331595 0.         0.11566272
 0.11566272 0.         0.         0.         0.         0.
 0.         0.11566272 0.         0.         0.         0.
 0.11566272 0.11566272 0.         0.         0.23132544 0.
 0.07746067 0.         0.         0.         0.

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [34]:
from scipy.spatial import distance
similarity_matrix = []
for i in range(len(tfidf_vectors)):
    row = []
    for j in range(len(tfidf_vectors)):
        similarity = 1 - distance.cosine(tfidf_vectors[i], tfidf_vectors[j])
        row.append(similarity)
    similarity_matrix.append(row)

df_similarity = pd.DataFrame(similarity_matrix, index=random_names, columns=random_names)

print('Близость между каждой парой рецептов:')
print(df_similarity.to_string())

Близость между каждой парой рецептов:
                           sausage quiche squares  cappuccino ice  wake up waffle sandwiches  chicken paprikash  mexican carne asada tacos
sausage quiche squares                   1.000000             0.0                   0.052920           0.022224                        0.0
cappuccino ice                           0.000000             1.0                   0.000000           0.000000                        0.0
wake up waffle sandwiches                0.052920             0.0                   1.000000           0.079965                        0.0
chicken paprikash                        0.022224             0.0                   0.079965           1.000000                        0.0
mexican carne asada tacos                0.000000             0.0                   0.000000           0.000000                        1.0


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

Наиболее похожими являются рецепты “texas chili” и “baked spanish risotto”, т.к. значение близости между ними является наибольшим значением в таблице (0.222029). Описания этих рецептов имеют наибольшее сходство среди всех выбранных рецептов.